In [1]:
# write all code in one cell 

#========================Load data=========================
import numpy as np
import pandas as pd

train_data_source = 'yelp_full_train.csv'
test_data_source = 'yelp_full_valid.csv'

train_df = pd.read_csv(train_data_source)
test_df = pd.read_csv(test_data_source)

In [2]:
# to change use .astype() 
test_df['text'] = test_df.text.astype(str)
train_df['text'] = train_df.text.astype(str)

In [5]:
train_df.head()
#test_df.dtypes

,label,text
0,3,Best Irish food in the Burgh. Great bar food ...
1,2,3/6/12 visit - i am from san francisco bay a...
2,2,I agree with Paul on this one. But I had to st...
3,3,"When I moved to the area, Dr. Weinberg was rec..."
4,1,"I love Steak N Shake. This one, however, leave..."


In [6]:
# convert string to lower case 
train_texts = train_df['text'].values 
train_texts = [s.lower() for s in train_texts] 

test_texts = test_df['text'].values 
test_texts = [s.lower() for s in test_texts] 

In [7]:
#=======================Convert string to index================
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)

# Convert string to index 
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
# Padding
train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
test_data = pad_sequences(test_texts, maxlen=1014, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')

In [9]:
#=======================Get classes================
train_classes = train_df['label'].values
train_class_list = [x-1 for x in train_classes]

test_classes = test_df['label'].values
test_class_list = [x-1 for x in test_classes]

from keras.utils import to_categorical
train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)

In [10]:
print(tk.word_index)

{'7': 52, '+': 55, 'o': 6, ']': 62, ')': 36, '-': 31, 'z': 33, '/': 46, 'i': 7, '\\': 27, '^': 64, 'y': 18, ',': 26, '.': 22, 'c': 15, 'v': 24, '9': 48, '{': 67, 'n': 8, '!': 29, '#': 58, '3': 44, '"': 35, 'm': 17, '*': 53, '$': 43, '=': 56, '5': 41, 'u': 14, '@': 61, 'g': 20, '0': 34, 'd': 12, 'h': 11, '&': 49, 's': 9, '?': 42, 'a': 5, '_': 59, 'w': 16, 'UNK': 1, 'x': 30, '}': 66, '1': 39, '4': 47, 'b': 23, 'f': 19, '8': 50, ';': 54, '%': 57, '(': 37, ' ': 2, 't': 4, 'k': 25, 'l': 13, '|': 65, '`': 68, 'q': 38, "'": 28, '[': 63, ':': 45, 'p': 21, '6': 51, '~': 60, 'e': 3, '2': 40, 'j': 32, 'r': 10}


In [11]:
vocab_size=len(tk.word_index)
vocab_size

68

In [12]:
embedding_weights=[]

In [13]:
embedding_weights.append(np.zeros(vocab_size))

In [14]:
for char, i in tk.word_index.items(): # from index 1 to 70
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)

In [15]:
print(embedding_weights.shape) # first row all 0 for PAD, 69 char, last row for UNK
embedding_weights

(69, 68)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [17]:
# parameter 
input_size = 1014
# vocab_size = 68
embedding_size = 68
conv_layers = [[256, 7, 3], 
               [256, 7, 3], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 3
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

In [18]:
# Embedding layer Initialization
embedding_layer = Embedding(vocab_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

In [19]:
# Model 

# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding 
x = embedding_layer(inputs)
# Conv 
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x) 
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x) # Final shape=(None, 34, 256)
x = Flatten()(x) # (None, 8704)
# Fully connected layers 
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x) # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) # Adam, categorical_crossentropy
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1014)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1014, 68)          4692      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1008, 256)         122112    
_________________________________________________________________
activation_1 (Activation)    (None, 1008, 256)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 336, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 330, 256)          459008    
_________________________________________________________________
activation_2 (Activation)    (None, 330, 256)          0         
__________

In [20]:
# 1000 training samples and 100 testing samples
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices][:]
y_train = train_classes[indices][:]

x_test = test_data[:]
y_test = test_classes[:]

In [22]:
# Training
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=15,
          verbose=2)

Train on 12434 samples, validate on 3109 samples
Epoch 1/15
 - 471s - loss: 0.6510 - acc: 0.6692 - val_loss: 0.7475 - val_acc: 0.6288
Epoch 2/15
 - 464s - loss: 0.5616 - acc: 0.7209 - val_loss: 0.7450 - val_acc: 0.6529
Epoch 3/15
 - 449s - loss: 0.4969 - acc: 0.7582 - val_loss: 1.1652 - val_acc: 0.5664
Epoch 4/15
 - 448s - loss: 0.4128 - acc: 0.8102 - val_loss: 0.8725 - val_acc: 0.6555
Epoch 5/15
 - 465s - loss: 0.3400 - acc: 0.8499 - val_loss: 0.9578 - val_acc: 0.6282
Epoch 6/15
 - 484s - loss: 0.2602 - acc: 0.8942 - val_loss: 1.5245 - val_acc: 0.5629
Epoch 7/15
 - 468s - loss: 0.2317 - acc: 0.9090 - val_loss: 1.4109 - val_acc: 0.6250
Epoch 8/15
 - 437s - loss: 0.1955 - acc: 0.9226 - val_loss: 1.4194 - val_acc: 0.6594
Epoch 9/15
 - 436s - loss: 0.1224 - acc: 0.9564 - val_loss: 1.5208 - val_acc: 0.6565
Epoch 10/15
 - 439s - loss: 0.0810 - acc: 0.9727 - val_loss: 2.1005 - val_acc: 0.6610
Epoch 11/15
 - 452s - loss: 0.0936 - acc: 0.9683 - val_loss: 1.6030 - val_acc: 0.6526
Epoch 12/15
 -

In [23]:
model.save("my_model_yelp_pol.h5")